In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pandas
import utils
from data_processing_bureaucrat.Bureaucrat import Bureaucrat
from pathlib import Path
import plotly.express as px
import measurements_table
from sweeping_voltage_scan_plots import get_color

Before starting, we build a table relating each scan name with each "voltage sweep measurement"

In [ ]:
measurements_table_df = measurements_table.create_measurements_table()

# For each of the "scan 1D seeping bias voltage" I collect all the single voltage 1D scans that were created in such "voltage sweep scan".
scans_and_sub_measurements_df = pandas.DataFrame()
for root_measurement_name in measurements_table_df.query('Type=="scan 1D sweeping bias voltage"').index:
    with open(utils.path_to_measurements_directory/Path(root_measurement_name)/Path('scan_1D_sweeping_bias_voltage/README.txt'), 'r') as ifile:
        for idx, line in enumerate(ifile):
            if idx == 0:
                continue
            scans_and_sub_measurements_df = scans_and_sub_measurements_df.append({'Scan name': line.replace('\n',''), 'Voltage scan measurement name': root_measurement_name}, ignore_index=True)
scans_and_sub_measurements_df.set_index('Scan name', inplace=True)
display(scans_and_sub_measurements_df)

Read interpixel distance and calibration factor (previously calculated by the respective scripts) for each measurement:

In [ ]:
time_resolutions_df = []#pandas.DataFrame()
for measurement_name in measurements_table_df.query('Type=="scan 1D"').index:
    try:
        this_measurement_time_resolution_df = utils.read_previously_calculated_time_resolution(measurement_name)
    except FileNotFoundError:
        #print(f'Cannot find time resolution calculation for measurement {measurement_name}...')
        continue
    try:
        this_measurement_calibration_factor = utils.read_previously_calculated_distance_calibration_factor(measurement_name)
    except FileNotFoundError:
        this_measurement_calibration_factor = float('NaN')
        print(f'Cannot find distance calibration factor for measurement {measurement_name}...')
    try:
        this_measurement_belongs_to_the_voltage_scan = scans_and_sub_measurements_df.loc[measurement_name,'Voltage scan measurement name']
    except KeyError:
        this_measurement_belongs_to_the_voltage_scan = '?'
    this_measurement_time_resolution_df['Distance calibration factor'] = this_measurement_calibration_factor
    this_measurement_time_resolution_df['Voltage scan measurement name'] = this_measurement_belongs_to_the_voltage_scan
    time_resolutions_df.append(this_measurement_time_resolution_df)#, ignore_index = True)
time_resolutions_df = pandas.concat(time_resolutions_df, axis=0, ignore_index=True)
display(time_resolutions_df)

Now build a full dataframe with this information and the information of each detector:

In [ ]:
time_resolutions_df.set_index('Measurement name', inplace=True)
time_resolutions_df['Device'] = measurements_table_df['Measured device']
time_resolutions_df['Bias voltage (V)'] = measurements_table_df['Bias voltage (V)']
time_resolutions_df.loc[time_resolutions_df['Bias voltage (V)']=='?','Bias voltage (V)'] = 'NaN'
time_resolutions_df['Bias voltage (V)'] = time_resolutions_df['Bias voltage (V)'].astype(float)
time_resolutions_df.reset_index(inplace=True)
time_resolutions_df.set_index('Device', inplace=True)
time_resolutions_df = time_resolutions_df.join(utils.bureaucrat.devices_sheet_df)

display(time_resolutions_df)
print(sorted(time_resolutions_df.columns))

# Plots

## Time resolution vs distance vs bias voltage

In [ ]:
df_to_plot = time_resolutions_df.reset_index().query('`Voltage scan measurement name`=="20211108201239_#20_sweeping_bias_voltage"').copy() # Select a single voltage sweep to show.
df_to_plot = df_to_plot.query('`k_1 (%)`==50').query('`k_2 (%)`==50')
df_to_plot = df_to_plot.query('`Bias voltage (V)`<230') # Higher values produce amplitudes > 1 V and enter into the nonlinear regime of the Cividec amplifiers.
df_to_plot = df_to_plot.sort_values(by=['n_position','Bias voltage (V)'])
fig = utils.line(
    data_frame = df_to_plot,
    line_group = 'Measurement name',
    x = 'Distance (m)',
    y = 'Time resolution (s)',
    line_dash = 'Pad',
    color = 'Bias voltage (V)',
    color_discrete_sequence = [get_color('Bluered', i/(len(set(df_to_plot['Bias voltage (V)']))-1)) for i in range(len(set(df_to_plot['Bias voltage (V)'])))],
    hover_name = 'Measurement name',
    title = f'Time res3olution vs scan distance @ different bias voltages<br><sup>Measurement: {list(set(df_to_plot["Voltage scan measurement name"]))[0]}</sup>',
    log_y = True,
    labels = {
        'Distance (m)': 'Laser position (m)',
    },
)
fig.show()
fig.write_html('time_resolution_vs_distance_vs_bias_voltage.html', include_plotlyjs = 'cdn')

## Average time resolution (in distance) vs bias voltage

In [ ]:
left_plateau_rows = (time_resolutions_df['Distance (m)']>66e-6)&(time_resolutions_df['Distance (m)']<133e-6)&(time_resolutions_df['Pad']=='left')
right_plateau_rows = (time_resolutions_df['Distance (m)']>200e-6)&(time_resolutions_df['Distance (m)']<266e-6)&(time_resolutions_df['Pad']=='right')
df = time_resolutions_df[left_plateau_rows|right_plateau_rows].copy()
df = df.query('`Time resolution (s)`<1e-9') # There are a few noisy points, just get rid of them.
df = df.query('`k_1 (%)`==50').query('`k_2 (%)`==50')
df.reset_index(inplace=True)
df = df[~df['Device'].isin({'1','2'})]
df = df[~df['Voltage scan measurement name'].isin(
    {
        '20211024163128_#65_sweeping_bias_voltage', # This measurement did not have all the pads grounded.
        '?',
    }
)]
df = df.query('`Bias voltage (V)`<=222') # Higher values produced amplitudes > 1 V that goes out the linear dynamic range of the amplifiers.
df = df[[
  #  '#', 
    'Bias voltage (V)', 
  #  'Delta_t std (s)', 
    'Device', 
  #  'Distance (m)', 
  #  'Distance calibration factor', 
    'Measurement name', 
  #  'PGAIN dose', 
  #  'Pad', 
    'Time resolution (s)', 
    'Voltage scan measurement name', 
  #  'color', 
  #  'comments', 
    'contact type', 
  #  'fluence', 
  #  'gain', 
  #  'irradiate with', 
  #  'k_1 (%)', 
  #  'k_2 (%)', 
  #  'n_position', 
  #  'n_trigger', 
    'pixel border', 
  #  'production', 
  #  'seems dead?', 
  #  'status', 
  #  'substrate thickness (µm)', 
    'trench depth', 
  #  'trench process', 
    'trenches', 
  #  'type', 
    'wafer', 
  #  'was measured before irradiation', 
  #  'x', 'y',
]]
df = utils.mean_std(
    df,
    by = list(set(df.columns).difference({
       'Time resolution (s)',
    }))
)
df = df.sort_values(by=['Bias voltage (V)','trenches','trench depth']).reset_index()
fig = utils.line(
    data_frame = df,
    line_group = 'Voltage scan measurement name',
    x = 'Bias voltage (V)',
    y = 'Time resolution (s) mean',
    error_y = 'Time resolution (s) std',
    error_y_mode = 'band',
    facet_col = 'wafer',
    facet_row = 'trenches',
    color = 'trench depth',
    symbol = 'pixel border',
    line_dash = 'contact type',
    grouped_legend = True,
    hover_name = 'Measurement name',
    text = 'Device',
    log_y = True,
    labels = {
        'Time resolution (s) mean': 'Time resolution (s)',
    },
)
fig.show()
fig.write_html('time_resolution_vs_bias_voltage.html', include_plotlyjs = 'cdn')